<a href="https://colab.research.google.com/github/kavyajeetbora/modern_geospatial_stack/blob/master/notebooks/metro_accessibility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q osmnx
!pip install -q pandana
!pip install -q pygaul
!pip install -q osmium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 995.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.7/553.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00


In [47]:
import pygaul
import ee
import geemap
import geopandas as gpd
import pandas as pd
import osmium
import osmnx as ox
import shapely

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

In [69]:
admin = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')

## Mumbai
Mumbai = admin.filter(ee.Filter.eq('ADM2_NAME', 'Mumbai city'))
Mumbai_suburban = admin.filter(ee.Filter.eq('ADM2_NAME', 'Mumbai Suburban'))
Mumbai = Mumbai_suburban.merge(Mumbai)

## Delhi
Delhi = admin.filter(ee.Filter.eq('ADM2_NAME', 'Delhi'))

## Bangalore
Bangalore = admin.filter(ee.Filter.eq('ADM2_NAME', 'Bangalore Urban'))

In [70]:
Map = geemap.Map()
Map.addLayer(ee_object=Mumbai, vis_params={'color':'red'}, name='Delhi')
Map.centerObject(Mumbai, zoom=10)
Map

Map(center=[19.120278006108997, 72.87471842263231], controls=(WidgetControl(options=['position', 'transparent_…

Converting `ee.Geometry` to `geopandas.GeoDataFrame`

In [98]:
def extract_polygons(gdf):
    polygons = []
    for geom_collection in gdf['geometry']:
        for geom in geom_collection.geoms:
            if isinstance(geom, shapely.geometry.polygon.Polygon):
                polygons.append(geom)


    geometry = shapely.unary_union(polygons)

    return geometry

In [99]:
## Delhi
Delhi = admin.filter(ee.Filter.eq('ADM2_NAME', 'Delhi'))
geemap.ee_to_gdf(Delhi)

,geometry,ADM0_CODE,ADM0_NAME,ADM1_CODE,ADM1_NAME,ADM2_CODE,ADM2_NAME,DISP_AREA,EXP2_YEAR,STATUS,STR2_YEAR,Shape_Area,Shape_Leng
0,"POLYGON ((76.83180 28.58173, 76.83853 28.55703...",115,India,1489,Delhi,17633,Delhi,NO,3000,Member State,1000,0.138635,2.064431


In [72]:
Mumbai_gdf = geemap.ee_to_gdf(Mumbai)
Mumbai_gdf

,geometry,ADM0_CODE,ADM0_NAME,ADM1_CODE,ADM1_NAME,ADM2_CODE,ADM2_NAME,DISP_AREA,EXP2_YEAR,STATUS,STR2_YEAR,Shape_Area,Shape_Leng
0,GEOMETRYCOLLECTION (LINESTRING (72.90397 18.99...,115,India,1498,Maharashtra,70183,Mumbai Suburban,NO,3000,Member State,1997,0.035050,1.221747
1,"GEOMETRYCOLLECTION (POINT (72.82761 18.90253),...",115,India,1498,Maharashtra,70184,Mumbai city,NO,3000,Member State,1997,0.006605,0.794985


In [118]:
dfx = gpd.GeoDataFrame(extract_polygons(Mumbai_gdf).geoms)
dfx['area'] = dfx[0].apply(lambda x: x.area)
dfx = dfx.sort_values(by='area', ascending=False)
mumbai_geom = dfx[0].iloc[0]

Mumbai_gdf = Mumbai_gdf.drop(0,axis=0)
Mumbai_gdf.loc[1, 'geometry'] = mumbai_geom

,geometry,ADM0_CODE,ADM0_NAME,ADM1_CODE,ADM1_NAME,ADM2_CODE,ADM2_NAME,DISP_AREA,EXP2_YEAR,STATUS,STR2_YEAR,Shape_Area,Shape_Leng
1,"POLYGON ((72.79168 18.94520, 72.79168 18.94744...",115,India,1498,Maharashtra,70184,Mumbai city,NO,3000,Member State,1997,0.006605,0.794985


In [85]:
geometry = shapely.unary_union(boundary['geometry'])
print(geometry)

MULTIPOLYGON (((72.80066364098649 18.88231422087147, 72.80290941954888 18.884559985195565, 72.80290941954888 18.882314210235418, 72.80066364098649 18.880068480316798, 72.80066364098649 18.88231422087147)), ((72.79168059208037 18.940703878771277, 72.79392636252199 18.94070390482242, 72.79392636252199 18.938458121463025, 72.79168059208037 18.938458115606007, 72.79168059208037 18.940703878771277)), ((72.80740091504025 19.075449218200248, 72.80740091504025 19.077694989387474, 72.80964672211518 19.077694975804945, 72.80964672211518 19.07544918446851, 72.80740091504025 19.075449218200248)), ((72.78269757610649 19.133838844551224, 72.78269757610649 19.13159314120477, 72.78045188370152 19.131593090018765, 72.78045188370152 19.133838860882594, 72.78045188370152 19.136084644771998, 72.78269757610649 19.136084648607806, 72.78269757610649 19.133838844551224)), ((72.78718907993422 19.18324547321802, 72.78718907993422 19.180999708354936, 72.78494333824767 19.180999711067848, 72.78718907993422 19.183